# Мэтчинг товаров дилера и заказчика

## Описание проекта

Заказчик производит несколько сотен различных товаров бытовой и промышленной химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь, занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках. Для оценки ситуации, управления ценами и бизнесом в целом, заказчик периодически собирает информацию о том, как дилеры продают их товар. Для этого они парсят сайты дилеров, а затем сопоставляют товары и цены. Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт заказчик. Например, могут добавляться новый слова (“универсальный”, “эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с товарами производителя делается вручную.  
Основная идея - предлагать несколько товаров заказчика, которые с наибольшей вероятностью соответствуют размечаемому товару дилера. Предлагается реализовать это решение, как онлайн сервис, открываемый в веб- браузере. Выбор наиболее вероятных подсказок делается методами машинного обучения

**ЦЕЛЬ:** разработать решения, которое автоматизирует процесс сопоставления товаров.  

**Задачи:**
   - выгрузить данные
   - сделать предобработку
   - обработать текст в столбцах с названиями товаров
   - создать ембеддинги предложений 
   - попробовать разные модели,
   - оценить метрики и выбрать лучшую модель


## Описание данных

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые данные:  

1. `dealers` - список дилеров:
   - id - уникальный ключ дилера;
   - name - наименование дилера</br>
</br>  
2. `dealer_products` - результат работы парсера площадок дилеров:
   - product_key - уникальный номер позиции;
   - price - цена;
   - product_url - адрес страницы, откуда собраны данные;
   - product_name - заголовок продаваемого товара;
   - date - дата получения информации;
   - dealer_id - идентификатор дилера (внешний ключ к dealers)</br>
</br>
3. `products` - список товаров, которые производит и распространяет заказчик:
   - id - уникальный ключ товара в базе заказчика
   - article - артикул товара;
   - ean_13 - код товара (см. EAN 13)
   - name - название товара;
   - cost - стоимость;
   - recommended_price - рекомендованная цена;
   - category_id - категория товара;
   - ozon_name - названиет товара на Озоне;
   - name_1c - название товара в 1C;
   - wb_name - название товара на Wildberries;
   - ozon_article - описание для Озон;
   - wb_article - артикул для Wildberries;
   - ym_article - артикул для Яндекс.Маркета;</br>  
</br>  
4. `match` - таблица матчинга товаров заказчика и товаров дилеров:
   - key - внешний ключ к dealer_products;
   - product_id - внешний ключ к products;
   - dealer_id - внешний ключ к dealers.

In [ ]:
import nltk
import spacy
import re 
import string
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.spatial.distance import pdist
from concurrent import futures
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 500)

## Выгрузка и обработка данных

In [ ]:
# Выгрузим данные из 4х таблиц в отдельные датафреймы
dealers = pd.read_csv('marketing_dealer.csv', sep=';')
dealer_products = pd.read_csv('marketing_dealerprice.csv', sep=';')
products = pd.read_csv('marketing_product.csv', sep=';')
match = pd.read_csv('marketing_productdealerkey.csv', sep=';')

In [ ]:
dealers.sample(5)

In [ ]:
dealers.info()

In [ ]:
dealers['name'].unique()

В таблице `dealers` собрана информация по названиям дилеров и их id. Всего в таблице представлено 18 уникальных дилеров.  
Для решения поставленной задачи на текущий момент данная таблица не требуется.

In [ ]:
dealer_products.sample(5)

In [ ]:
# удалим лишний столбец 'id
dealer_products = dealer_products.drop(['id'], axis=1)

In [ ]:
dealer_products.info()

In [ ]:
dealer_products.nunique()

In [ ]:
sorted(dealer_products['date'].unique())

В таблице из 20416 записей лишь около 10% уникальных ключей, ссылок и названий продуктов. Все данные собраны за 14 дней: с 11-07 по 31-0-23.

In [ ]:
dealer_products.isna().sum()

В столбце product_url имеются пропуски, исследуем его подробнее.

In [ ]:
no_url = dealer_products[dealer_products['product_url'].isna()]['dealer_id'].unique()
no_url

In [ ]:
dealers[dealers['id'] == no_url[0]]

In [ ]:
dealer_products[dealer_products['dealer_id'] == 7]

Все имеющиеся в таблице пропуски относятся к дилеру под номером 7 - Комус.

In [ ]:
# Проверим длину названий продуктов для определения неявных пропусков.
dealer_products['product_name'].str.len().min(), dealer_products['product_name'].str.len().max()

In [ ]:
dealer_products[dealer_products['product_name'].str.len() == 8]

In [ ]:
# проверим данные на дубликаты
dealer_products.duplicated().sum()

In [ ]:
duplicates = dealer_products.duplicated()
dealer_products[duplicates].sort_values(by='product_name').head(5)

In [ ]:
# уберём дубликаты по столбцам: ключ, url и названию
dealer_products.drop_duplicates(subset=['product_key', 'product_url', 'product_name'], inplace=True)

In [ ]:
dealer_products.duplicated(subset=['product_key', 'product_url', 'product_name']).sum()

In [ ]:
# сбросим индексы
dealer_products.reset_index(drop = True, inplace = True)

В таблице `dealer_products` 20416 записей.  
Имеются пропуски в столбце `product_url` - 234 записи и все для дилера с id 7 - Komus. Полных дублей в таблице нет, но есть повторяющиейся записи в зависимости от даты выгрузки. 
Все столбцы имеют правильный тип, кроме даты, в рамках проекта дату приводить к нужному формату нет необходимости.  
Столбец `product_key` содержит данные в текстового типа, он состоит из ключей не только в виде числа, но и в виде ссылок на сайты с продуктами.   

Столбец `product_name` является целевым: по нему будем находить соответствие продуктов из базы заказчика.  
В названиях имеются как слова на кириллице, так и на латинице; есть специальные символы, единицы измерения разные: кг, л, мл; в некоторых названиях в конце указан код, состоящий из цифр и "-"; попадаются сокращения (например: дер. конструкций, д/удаления), в рамках одного названия встречаются буквы в разных регистрах.

In [ ]:
# удалим лишний столбец 'Unnamed: 0'
products = products.drop(['Unnamed: 0'], axis=1)

In [ ]:
products.sample(5)

In [ ]:
products.info()

In [ ]:
products.nunique()

In [ ]:
# удалим столбец wb_article_td так как он содержит мало записей и не содержит важной информации.
products.drop(['wb_article_td'], axis=1, inplace=True)

In [ ]:
products.isna().sum()

In [ ]:
mask = products['name'].isna()
products[mask]

In [ ]:
#удалим строки, где пропуски в названиии товара
products.dropna(subset=['name'], inplace=True)
products.reset_index(drop=True, inplace= True)

In [ ]:
mask = products['cost'].isna()
products[mask]

In [ ]:
products.duplicated().sum()

In [ ]:
# изучим наименования продуктов детальнее
products[['name', 'name_1c']].sample(5)

На первый взгляд в названиях из 1С меньше лишней или технической информации, меньше опечаток.

In [ ]:
# проверим минимальную и максимальную длину названия
products['name'].str.len().min(), products['name'].str.len().max()

In [ ]:
products[products['name'].str.len() == 3]
# в данной строке отсутствует название, запись можно удалить

In [ ]:
i = products[products['name'].str.len() == 3].index
products.drop(i, inplace = True)
products.reset_index(drop = True, inplace = True)

In [ ]:
products.info()

В таблице `products` 496 записей.  
Имеются 2 записи, где отсутствует большая часть информации.  
Для товаров *Герметик акриловой цвет Белый, 7 кг; Герметик акриловый цвет Медовый 0,6 л (12 шт); Герметик акриловый цвет Орех, ф/п 600мл (12 штук)* отсутствуют стоимость и рекомендованная цена.
Дубликаты отсутствуют.  

Для построения модели мэчинга можем использовать данные в столбце `name` или `name_1c`, всего имеется 487 уникальных наименований.
В названиях имеются опечатки, лишние пробелы, специальные символы, иногда отсутствуют пробелы между словами: часто сливаются слова на кириллице и латинице. В части продуктов указана рекомендуемая концентрация, для некоторых продуктов указан вес (в кг.), а для других объём (в мл. или л.). Концентрация, вес или количество обычно указываются в конце названия. В рамках одного названия встречаются буквы в разных регистрах. Максимальная длина наименования продукта 136 символов, минимальная - 30.

In [ ]:
# удалим лишний столбец 'Unnamed: 0'
match = match.drop(['id'], axis = 1)

In [ ]:
match.sample(5)

In [ ]:
match.info()

In [ ]:
match.nunique()

In [ ]:
match.sort_values(by='key', ascending=False)

In [ ]:
match.duplicated().sum()

В таблице `match` 1700 записей, кол-во уникальных id дилеров совпадает с количеством в таблице `dealers`. Пропусков нет, дубликатов тоже. Столбец `key` имеет текстовый формат, в нём есть записи не только уникальных ключей, но и url продуктов.  
Данная таблица может пригодиться, когда будем оценивать эффективность мэтчинга.

## Предобработка названий

In [ ]:
# заполним пропуски в name_1c данными из столбца name
products['name_1c'].fillna(products['name'], inplace=True)

В дальнейшем будем работать со столбцами `1c_name` из таблицы `products` и `product_name` из `dealer_products`.
Необходимо произвести предобработку текста, чтобы названия в обеих таблицах были наиболее схожи, для этого необходимо:
1. убрать лишние пробелы
2. привести к нижнему регистру
3. добавить пробелы между русскими словами и английскими: как до, так и после
4. убрать концентрацию, оставить только объём/вес
5. убрать стоп-слова

In [ ]:
# функция для базовой обработки текста
def clean_text(text):
    #добавляем пробелы между русскими и английскими словами
    #pattern = re.compile(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])')
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)
    #убираем указание концентрации
    #pattern2 = re.compile(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*')
    text = re.sub(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*', '', text)
    #убираем специальные символы
    remove = string.punctuation
    #remove = remove.replace("-", "") # не убираем дефисы
    text = re.sub('[%s]' % re.escape(remove), ' ', text)
    # убираем лишние пробелы между словами
    text = re.sub(r'\s+', ' ', text)
    #приводим все слова к нижнему регистру
    #text = text.lower()
    return text

# функция убирает служебные слова и лемматизирует текст
def preprocess_text(text):
    # удаление одиноко стоящих слов
    #text = re.sub(r'\s+[a-zA-Zа-яА-Я0-9]\s+', ' ', text)
    #Очистка текста 
    text = re.sub(r"[^a-zA-Zа-яА-ЯёЁ ]", ' ', text)
    #doc = nlp(text)
    #токенизация, лемматизация 
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    #удаление стоп-слов
    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    lemmas_clean = [lemma for lemma in lemmas if lemma not in stop_words]
    #tokens = [token.lemma_ for token in doc]
    #tokens = [token for token in tokens if token not in stop_words and token != " "]
    #text = " ".join(tokens)
    return " ".join(lemmas_clean)

# функция выделеяет единицы измерения из текста
def extract_measure(text):
    measurements = []
    pattern = r'\s*(\d+(?:[,.]\d+)?)\s?[л|мл|кг]+'
    match = re.search(pattern, text)
    if match:
        measurements =  match.group(1)
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    else:
        measurements = 0
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    return measurements

# функция вовращает список длин
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

In [ ]:
# создадим столбцы с единицами измерения товара в обеих таблицах
products['measures'] = products['name_1c'].apply(extract_measure)
products['measures'] = products['measures'].str.replace(',', '.')
products['measures'].fillna(0, inplace=True)
products['measures'] = products['measures'].astype(float)

dealer_products['measures'] = dealer_products['product_name'].apply(extract_measure)
dealer_products['measures'] = dealer_products['measures'].str.replace(',', '.')
dealer_products['measures'].fillna(0, inplace=True)
dealer_products['measures'] = dealer_products['measures'].astype(float)

In [ ]:
products.sample(5)

In [ ]:
#создадим столбцы с длиной названия товара
products['name_length'] = get_text_length(np.array(products['name_1c']))
dealer_products['name_length'] = get_text_length(np.array(dealer_products['product_name']))

In [ ]:
%%time
# создадим новый столбец marketing_name - он включает в себя все названия из 1с
products['marketing_name'] = products['name_1c'].apply(clean_text)
#pattern = r'\b(\d+)\s?[л|мл|кг]+'
products['marketing_name'] = products['marketing_name'].str.replace(r'\b(\d+)\s?[л|мл|кг]+', '')

In [ ]:
products[['name_1c','marketing_name']].sample(3)

In [ ]:
%%time
# лемматизируем текст
#nlp = spacy.load("ru_core_news_lg")
products['marketing_name'] = products['marketing_name'].apply(preprocess_text)

In [ ]:
products[['name_1c','marketing_name']].sample(3,random_state=1)

In [ ]:
dealer_products['product_name'].head()

In [ ]:
%%time
# аналогичным образом обработаем столбец product_name
dealer_products['dealer_name'] = dealer_products['product_name'].apply(clean_text)
dealer_products['dealer_name'] = dealer_products['dealer_name'].str.replace(r'\b(\d+)\s?[л|мл|кг]+', '')
dealer_products['dealer_name'].head()

In [ ]:
%%time
dealer_products['dealer_name'] = dealer_products['dealer_name'].apply(preprocess_text)
dealer_products['dealer_name'].head()

In [ ]:
# объединим id, marketing_name, product_key, dealer_name с таблицей match
match_products = match[['key', 'product_id']].merge(
    products[['id', 'marketing_name']],  how='left', right_on='id', left_on='product_id')

# к датасету с названиями диллеров присоединим названия производителя 
df = dealer_products[['product_key', 'dealer_name']].merge(
    match_products, how='left', right_on='key', left_on='product_key').drop(['product_id', 'product_key'], axis=1)

df = df.dropna().reset_index(drop=True)
df

## Векторизация текста и поиск мэтчей

### SBERT

In [ ]:
# %%time
# model = SentenceTransformer('all-MiniLM-L6-v2')

# products = list(df['marketing_name'])
# dealers = df['dealer_name']
# prod_embeddings = model.encode(products)
# deal_embeddings = model.encode(dealers)

In [ ]:
# top_k = 5
# cos_scores = util.cos_sim(deal_embeddings, prod_embeddings)
# top_results = torch.topk(cos_scores, k=top_k)
# print("Cosine-Similarity:", cos_sim)

In [ ]:
# x = util.semantic_search(prod_embeddings, deal_embeddings, top_k = top_k)
# x

In [ ]:
# match_key = []
# metrics = []
# for i in range(len(x[:3])):
#     for k in range(top_k):
#         match_key.append(df.iloc[x[i][k]['corpus_id'],:]['id'])
#         if df.iloc[i]['id'] in match_key[0]:
#             print('yes')

In [ ]:
# for query in dealers:
#     # We use cosine-similarity and torch.topk to find the highest 5 scores
#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     print("\n\n======================\n\n")
#     print("Query:", query)
#     print("\nTop 5 most similar sentences in corpus:")

#     for score, idx in zip(top_results[0], top_results[1]):
#         print(corpus[idx], "(Score: {:.4f})".format(score))

In [ ]:
# for i in range(len(x)):
#     print('===================')
#     print(f"Запрос: {df['dealer_name'][i]}")
#     print('===================')
#     for k in range(3):
#         #print(sentences[x[i][k]['corpus_id']])
#         print(f"Название продукта: {df['marketing_name'][x[i][k]['corpus_id']]}, оценка: {x[i][k]['score']}")

In [ ]:
# matches = []

# for col in set(match_df.columns):
        
#     top_cands = match_df.loc[:, col].sort_values(ascending=True)[:top].index.tolist()
#     product_key = ''.join(col.split('_')[:-1])
    
#     # print(product_key)
    
#     if  match.loc[match['key'] == product_key].shape[0] == 0:
#         matches.append(0)
#         continue
        
#     match_id = match.loc[match['key'] == product_key, 'product_id'].values[0]
#     if match_id in top_cands:
#         matches.append(1)
#     else:
#         matches.append(0)
        
# print(f'Значение метрики Accuracy@{top_k} рекомендаций: {np.mean(matches)}')

### Вариант2: 

In [ ]:
#Создадим словарь,где ключом являются id продукта, а значением - название
#marketing_name = pd.Series(products['marketing_name'].values, index=products['id']).to_dict()
# marketing_name = pd.Series(products['name_1c'].values, index=products['id']).to_dict()
# marketing_name

In [ ]:
#Создадим словарь,где ключом являются product_key дилера, а значением - название у дилера
#dealer_name = pd.Series(dealer_products['dealer_name'].values, index=dealer_products['product_key']).to_dict()
# dealer_name = pd.Series(dealer_products['product_name'].values, index=dealer_products['product_key']).to_dict()
# dealer_name

In [ ]:
%%time
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('distilbert-base-nli-mean-tokens') 

rows = df['marketing_name'].values
columns = df['dealer_name'].values
market_names = model.encode(rows)
dealer_names = model.encode(columns)

In [ ]:
# # дополним векторы названий данными об объёме/весе и длине строки
# X1 = pd.DataFrame(market_names)
# X1['measures'] = products['measures']
# X1['name_length'] = products['name_length']

# X2 = pd.DataFrame(dealer_names)
# X2['measures'] = dealer_products['measures']
# X2['name_length'] = dealer_products['name_length']

# # # отмасштабируем данные
# numeric = ['measures', 'name_length']
# scaler = MinMaxScaler()

# scaler.fit(X1[numeric])
# X1[numeric] = scaler.transform(X1[numeric])

# scaler.fit(X2[numeric])
# X2[numeric] = scaler.transform(X2[numeric])

# # # преобразуем данные в разряженную матрицу
# X1_sparse = sparse.csr_matrix(X1.values)
# X2_sparse = sparse.csr_matrix(X2.values)

In [ ]:
#создадим матрицу соответствий названий
# data = pairwise_distances(X1_sparse, X2_sparse, metric = 'cosine')
# match_df = pd.DataFrame(index = products['name_1c'], columns = dealer_products['product_name'], data=data)
# match_df

data = pairwise_distances(market_names, dealer_names, metric = 'cosine')
match_df = pd.DataFrame(index = products['id'], 
                        columns = dealer_products['product_key']+ '_' + pd.Series(range(dealer_products.shape[0])).astype(str), 
                        data=data)
match_df

In [ ]:
# рассчитаем Accuracy@k для оценки качества мэтчинга
top_k = 5
matches = []

for col in set(match_df.columns):
        
    top_cands = match_df.loc[:, col].sort_values(ascending=True)[:top_k].index.tolist()
    product_key = ''.join(col.split('_')[:-1])
    
    if  match.loc[match['key'] == product_key].shape[0] == 0:
        matches.append(0)
        continue
        
    match_id = match.loc[match['key'] == product_key, 'product_id'].values[0]
    if match_id in top_cands:
        matches.append(1)
    else:
        matches.append(0)
        
np.mean(matches)

### Вариант 3

In [ ]:
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
# from sklearn.preprocessing import FunctionTransformer
# from collections import defaultdict

# def get_text_length(x):
#     return np.array([len(t) for t in x]).reshape(-1, 1)

In [ ]:
# %%time
# corpus = pd.concat([products['marketing_name'], dealer_products['dealer_name']], axis = 0)
# count_tf_idf = TfidfVectorizer()
# corpus_vect = count_tf_idf.fit(corpus)
# df_1 = count_tf_idf.transform(products['marketing_name'])
# df_2 = count_tf_idf.transform(dealer_products['dealer_name'])

In [ ]:
# df_1 = df_1.todense() #df_1.toarray()
# df_2 = df_2.todense() #df_2.toarray()

In [ ]:
# matr = pd.DataFrame(data = pairwise_distances(df_1, df_2, 'cosine'), 
#              index = products['id'], 
#              columns = dealer_products['product_name'])

In [ ]:
# pipe = Pipeline([('count', CountVectorizer(ngram_range= (1,2))),
#                  ('tfid', TfidfTransformer())]).fit(corpus)
# X = pipe['count'].transform(corpus).toarray()
# X = pipe['tfid'].idf_
# X = pipe.transform(corpus)
# X